<a href="https://colab.research.google.com/github/simsime1209/-Did-you-make-this-fucking-anime-AI-/blob/main/NovelAILeaks_API_Backend_(4chan_Ver_)%EC%9D%98_%ED%95%9C%EA%B5%AD%EC%96%B4_%EB%B2%88%EC%97%AD%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

4chan의 NovelAILeaks(naifu)의 수정된 버전을 기반으로 합니다. [원본](https://boards.4channel.org/g/thread/89095460#p89097704)

공식 프런트엔드 + 최적화된 백엔드를 사용하여 75개 한계를 돌파하고 모든 모델을 지원할 수 있습니다.

크레딧: https://t.me/StableDiffusion_CN https://t.me/exlolicon

Thanks: Anonymous, 炼铜术士, 神楽坂早苗️, Jonathan, 咕 咕, 猫又逆变器, Gaein nidb

발번역 : simsime


*로컬로 운영 시 아나콘다 파워셀에 

jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0


In [ ]:
#@title ### 0. GPU 작동 상태 확인

!nvidia-smi

In [ ]:
#@title ### 1. Novel AI API 백엔드 및 모델 다운로드
#@markdown 다운로드 속도가 너무 느리다면 다시 시작해 보새요!

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

In [ ]:
#@title ### 2. 종속성 설치
#@markdown 설치가 완료될 때까지 기다리세요. 몇 분정도 소요됩니다!

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared
!/content/naifu/venv/bin/python -m pip install -qq pytorch_lightning==1.7.7

In [ ]:
#@title ### 3. 모델 시작
#@markdown 출력에 매핑된 주소로 새 창을 여세요. ('trycloudflare.com' / 'bore.pub'로 끝납니다 ;)  <- 서버가 열리면 접속할 주소
#@markdown - 모델 로딩이 완료될 때까지 기다린 후（`Application startup complete'가 나타나면 새로 열었던 창을 새로고침하세요! 서버가 활성화 된 거랍니다!
#@markdown - 'Cloudflare'에서 제공하는 서비스는 때때로 지연이 있을 수 있어서, 'bore' 터널을 대신 ​​사용할 수도 있어요.

%cd /content/naifu
!sed -i 's/# export 8⁹="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ### 4. (선택사항) 7G의 'animefull-latest' 모델로 실행
#@markdown 기본적으로 4G 크기의 animefull-final-pruned 모델이 사용됩니다. 7G의 animefull-최신 모델을 사용하고 싶다면 이 셀을 사용하세요!

%cd /content/
!mkdir /content/naifu/models/animefull-latest
!tar xf animefull-latest.tar -C /content/naifu/models/animefull-latest && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969